# Predicción de Suscripción a Depósito a Plazo Fijo - Campaña Telefónica

### Introduccion
En el contexto de la gestión de clientes y la optimización de campañas de marketing, los bancos buscan estrategias más efectivas para aumentar la contratación de sus productos financieros. Este proyecto se centra en el desarrollo de un modelo de machine learning capaz de predecir la probabilidad de que un cliente acepte un depósito a plazo fijo durante la campaña anual telefónica del banco. Utilizando los datos recopilados de los últimos tres años, la solución propuesta permitirá priorizar los clientes con mayor propensión a contratar el producto, optimizando recursos y mejorando los resultados de la campaña de manera eficiente.

### Objetivo
Desarrollar un modelo predictivo de Machine Learning que identifique con precisión qué clientes tienen mayor probabilidad de suscribir un depósito a plazo fijo durante la próxima campaña telefónica, permitiendo una asignación eficiente de recursos y maximizando el retorno de inversión, asegurando un AUC-ROC > 0.75 en validación.
